In [2]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_labeled = pd.read_csv('./data/base_files/labled_train_data.csv', index_col=0, sep='\t', comment='#')
tracks_unlabeled = pd.read_csv('./data/base_files/unlabled_train_tracks.csv', index_col=0, sep='\t', comment='#')

In [4]:
def sort_df(df):
    result =df
    result= df.sort_values(['order_id', 'dt']) # sort df by time
    result.index = range(len(df))
    return result
tracks_unlabled = sort_df(tracks_unlabeled)

In [14]:
tracks_unlabled

,driver_id,dt,lat_,lon_,order_id,speed
0,6bcc649b6ec22251179da12125d04011,2021-03-29 11:55:56,55.757886,37.406491,0000a57c86cabd27d707a5fde1d0fbe4,-1.678118
1,6bcc649b6ec22251179da12125d04011,2021-03-29 11:56:02,55.757886,37.406491,0000a57c86cabd27d707a5fde1d0fbe4,-3.379243
2,6bcc649b6ec22251179da12125d04011,2021-03-29 11:56:23,55.758017,37.406500,0000a57c86cabd27d707a5fde1d0fbe4,-2.969130
3,6bcc649b6ec22251179da12125d04011,2021-03-29 11:56:43,55.757987,37.406500,0000a57c86cabd27d707a5fde1d0fbe4,-3.324069
4,6bcc649b6ec22251179da12125d04011,2021-03-29 11:57:04,55.758015,37.406495,0000a57c86cabd27d707a5fde1d0fbe4,1.335598
...,...,...,...,...,...,...
674713,4cbab2104a47e4ea966c7f2ecd8f4775,2021-03-26 20:44:42,55.656228,37.494256,fffface895e65d8da177137701b1ee98,NaN
674714,4cbab2104a47e4ea966c7f2ecd8f4775,2021-03-26 20:45:04,55.656143,37.494459,fffface895e65d8da177137701b1ee98,NaN
674715,4cbab2104a47e4ea966c7f2ecd8f4775,2021-03-26 20:45:24,55.655037,37.496817,fffface895e65d8da177137701b1ee98,NaN
674716,4cbab2104a47e4ea966c7f2ecd8f4775,2021-03-26 20:45:46,55.654412,37.498307,fffface895e65d8da177137701b1ee98,NaN


Some functions for finding speed 

In [25]:
import datetime 
from math import cos, asin, sqrt, pi, pow

# formula using pythogorian theorem
# (as distances are not large, we can approximate earth rounding)
def get_distance(lat1,lon1,lat2,lon2):
    delta_lat = pow(lat2-lat1,2)
    delta_lon = pow(lon2-lon1,2)
    return np.round(sqrt(delta_lat+delta_lon)*100, 3)

def get_speed(lat1, lon1, lat2, lon2, dt1: str, dt2: str) -> float:
    distance = get_distance(lat1, lon1, lat2, lon2).tolist()
    format = "%Y-%m-%d %H:%M:%S"
    dt1=datetime.datetime.strptime(dt1, format)
    dt2=datetime.datetime.strptime(dt2, format)
    time = (dt2-dt1).total_seconds()/3600 # convert timedelta into hours
    if time==0:
        return 0
    return distance/time

code for building a ride route 

In [19]:
import requests
import json

def get_route(order: str, df_original: pd.DataFrame) -> dict:
    route = []
    df = df_original
    df = df[df['order_id']==order]
    for _, el in df.iterrows():
        point_info = [el['lon_'], el['lat_'], el['dt']]
        route.append(point_info)
    return {order: route}

api_key = '5b3ce3597851110001cf62485e43a8ce4c9147be87344cce215d4774'
def get_gpx_from_route(route:list, api_key: str):
    route = [x[:-1] for x in route] # remove time element
    url = 'https://api.openrouteservice.org/v2/directions/driving-car/gpx'
    gpx = requests.post(url, data=json.dumps({"coordinates":route}), headers = {'Authorization': api_key, 'content-type': 'application/json'})
    return gpx

order = '001662da857b5a39bb402aacf3145f86' # just for an example
route = get_route(order, tracks_labeled)
gpx = get_gpx_from_route(route[order], api_key)
with open('./route.gpx', 'w') as f:
    f.write(str(gpx.content))

In [26]:
def speed_to_df(df):
    result = df
    for i in range(1, len(df)):
        result.loc[i, 'speed'] = get_speed(df.loc[i-1, 'lat_'], df.loc[i-1, 'lon_'],
                                       df.loc[i, 'lat_'], df.loc[i, 'lon_'], df.loc[i-1, 'dt'], df.loc[i, 'dt'])
    return result

speed_unlabled = speed_to_df(tracks_unlabled)
speed_unlabled.to_csv('./data/unlabled_train_tracks_speed')


In [24]:
for i,order in enumerate(train_labeled['order_id']):
    tracks_labeled.loc[tracks_labeled['order_id']==order, 'is_aggressive'] = train_labeled.loc[i, 'is_aggressive']

KeyboardInterrupt: 